<a href="https://colab.research.google.com/github/anshupandey/Generative-AI-for-Professionals/blob/main/LangChain_Custom_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing necessary libraries
!pip install langchain-openai --quiet  # Installs a specific version (0.0.3) of the langchain_openai package quietly
!pip install langchain langchain-experimental --quiet        # Installs a specific version (0.1.1) of the langchain package quietly
!pip install openai wikipedia --quiet

## Custom Agent

### Load the LLM

In [ ]:
from langchain_openai import ChatOpenAI
# initializing llm with GPT 3.5 from OpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0,api_key="sk-xxxxxxxxxxxxxxxxxxxxxxxxxx")


### Defining custom Tools

In [ ]:
# Defining custom tool for the agent
from langchain.agents import tool
# A Python function to calculate the length of a word
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

get_word_length.invoke("abc")

3

In [ ]:
tools = [get_word_length]

### Create custom Prompts & Adding Memory

Now let us create the prompt. Because OpenAI Function Calling is finetuned for tool usage, we hardly need any instructions on how to reason, or how to output format. We will just have two input variables: input and agent_scratchpad. input should be a string containing the user objective. agent_scratchpad should be a sequence of messages that contains the previous agent tool invocations and the corresponding tool outputs.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
# set up a list to track the chat history
chat_history = []

### Bind tools to LLM

In [ ]:
# bind tools to LLLM
llm_with_tools = llm.bind_tools(tools)

### Create a Custom Agent

In [ ]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5The word "educa" is not a real word. It is a 5-letter word, but it does not have a specific meaning in the English language.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='There are 5 letters in the word "educa".')],
 'output': 'The word "educa" is not a real word. It is a 5-letter word, but it does not have a specific meaning in the English language.'}